### Rigid Body Dynamics

This problem will guide you to implement a basic physical simulator that simulates the motion of rigid bodies.

The tutorial starting from https://www.toptal.com/game/video-game-physics-part-i-an-introduction-to-rigid-body-dynamics is a perfect guide to help you understand what happens in the 2D rigid world. Now you need to implement a modified version in python.

#### Kinematics
We describe the kinematic state of a single object with a three-tuple $q=(x, y, \theta)$ where $(x, y)$ is the 2D coordinate of the object's center of mass and $\theta$ is its rotation. We assume each rigid body has a convex shape for simplicity, and use the vertex array of its convexhull at time steps $0$ to describe its geometry.

The code below provide you the trajectory of two rigid bodies, e.g., the states at each time step, you need to generate a video to see how they move.  

Finish the `RigidBody.get_shape` function in [here](rigid_body.py). You will see the motion of a triangle and a box.

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
from rigid_body import RigidBody, plot_trajectoreis, animate
    

objects = [RigidBody([(-0.3, -0.3), (0.5, 0.3), (-0.3, 0.8)], 'r'), 
           RigidBody([(-0.2, -0.2), (-0.2, 0.2), (0.2, 0.2), (0.2, -0.2)], 'g')]
n = len(objects)
cur = np.array([[-0.5, 0., 0.], [0.5, 0., 0.]])

next =np.random.random(size=(n, 3)) * 2  - 1
next[:, 2] *= np.pi
trajs = []
for j in range(10):
    trajs.append((next - cur) *(j+1)/10 + cur)


images = plot_trajectoreis(objects, trajs)
animate(images)

#### Single Object Dynamics 
Now you need consider the physics of a single object. In addition to the kinematic state $q$, we need another three dimension $\dot q$ to describe its velocity. Let $s=(q,\dot q)$ be a six dimension tuple, You need complete the function `RigidBody.step(h)` function that takes the current state $s$, wrench $w=(f_x, f_y, \tau)$ and time step $h$ as input, outputs the rigid body states at the next time steps. Note that you also need complete the function `apply_wrench` to apply force and torque to the objects. The first object will receive the gravity at its center of mass, while the second object recieve a force on the top-right corner. What will happen and why?

In [ ]:
objects = [RigidBody([(-0.2, -0.2), (-0.2, 0.2), (0.2, 0.2), (0.2, -0.2)], 'r', mass=100, inertia=10), 
           RigidBody([(-0.2, -0.2), (-0.2, 0.2), (0.2, 0.2), (0.2, -0.2)], 'g', mass=100, inertia=10)]

objects[0].set_state([-0.5, 0., 0., 0., 0., 0.])
objects[1].set_state([0.5, 0.0, 0., 0., 0., 0.])

h = 0.1
trajs = []
v_theta = []
for i in range(1000):
    # add gravity to the first objects
    objects[0].apply_wrench(objects[0].state[:2], [0, -10])
    objects[1].apply_wrench(objects[1].object2world([1, 1]), [0, -10])
    trajs.append([i.step(h) for i in objects])
    v_theta.append(objects[1].get_state()[2])

images = plot_trajectoreis(objects, trajs, width=4)
animate(images)

#### Multi-object Dynamics Without Friction
Now we need consider interactions between objects. The basic idea is that once two objects collides/contacts, there must be a force at the contact points to separate them so that they do not penetrate with each other.

##### Collision Detection

Before solving collisions, we first need to find them. The basic approach for detect collisions is the GJK+EPA algorithm which will return the contact points, together with the penetration depth, and the normal to separate the two shapes.

See https://www.toptal.com/game/video-game-physics-part-ii-collision-detection-for-solid-objects and https://dyn4j.org/2010/05/epa-expanding-polytope-algorithm/ for details.

The code below only considers the block stacking. The `plot_contacts` function helps you to plot the contact points together with the penetration direction. You need add test cases for more complex shapes (convex shape with $5$ points).

In [ ]:
from rigid_body import GJK, plot_contacts

objects = [RigidBody([(-0.1, -0.1), (-0.1, 0.1), (0.1, 0.1)], 'r', mass=1000, inertia=10), 
           RigidBody([(-0.2, -0.2), (-0.2, 0.2), (0.2, 0.2), (0.2, -0.2)], 'g', mass=1000, inertia=10)]
objects[0].set_state([0., 0., 0.])
objects[1].set_state([0., 0.3, np.pi/4])
plot_contacts(objects, GJK(objects[0].get_shape(), objects[1].get_shape(), plot=False))


objects = [RigidBody([(-0.1, -0.1), (-0.1, 0.1), (0.1, 0.1), (0.1, -0.1)], 'r', mass=1000, inertia=10), 
           RigidBody([(-0.2, -0.2), (-0.2, 0.2), (0.2, 0.2), (0.2, -0.2)], 'g', mass=1000, inertia=10)]
objects[0].set_state([0., 0., 0.])
objects[1].set_state([0., 0.3, 0.])
plot_contacts(objects, GJK(objects[0].get_shape(), objects[1].get_shape(), plot=False))


objects = [RigidBody([(-0.1, -0.1), (-0.1, 0.1), (0.1, 0.1), (0.1, -0.1)], 'r', mass=1000, inertia=10), 
           RigidBody([(-0.2, -0.2), (-0.2, 0.2), (0.2, 0.2), (0.2, -0.2)], 'g', mass=1000, inertia=10)]
objects[0].set_state([0., 0., 0.])
objects[1].set_state([0., 0.3, 0.])
plot_contacts(objects, GJK(objects[1].get_shape(), objects[0].get_shape(), plot=False))


objects = [RigidBody([(-0.1, -0.1), (-0.1, 0.1), (0.1, 0.1), (0.1, -0.1)], 'r', mass=1000, inertia=10), 
           RigidBody([(-0.2, -0.2), (-0.2, 0.2), (0.2, 0.2), (0.2, -0.2)], 'g', mass=1000, inertia=10)]
objects[0].set_state([0., 0., 0.])
objects[1].set_state([0., 0.2, 0.])
plot_contacts(objects, GJK(objects[1].get_shape(), objects[0].get_shape(), plot=False))



# TODO: add test cases for multiple objects and other convex shapes


##### Linear Complementray Problem 

Now you have a way to find all contacts, their directions and penetration distances, you need solve the contact forces at each contact points to separate them so that they will not penetrate in the next time step. If we ignore the Coulumb friction cone, this is the famous Linear Complementary Problem (https://en.wikipedia.org/wiki/Linear_complementarity_problem).

Please formulate the rigid body dynami as a LCP and solve it with either Lemake Algorithm or convex optimization methods.

You now need complete the following function `LCP(objects, contacts, h)` to compute the contact force under the external wrench. Note the wrenches are stored in each object class.

Run the following code to support block stacking under the gravity force. Besides, test your code with a newton ball-like experiments (where every adjecant ball contacts with each other initially), show what you find.

In [ ]:
objects = [
    RigidBody([(-0.1, -0.1), (-0.1, 0.1), (0.1, 0.1), (0.1, -0.1)], 'r', mass=1, inertia=10), 
    RigidBody([(-0.2, -0.2), (-0.2, 0.2), (0.2, 0.2), (0.2, -0.2)], 'g', mass=10, inertia=10),
    RigidBody([(-0.3, -0.3), (-0.3, 0.3), (0.3, 0.3), (0.3, -0.3)], 'b', mass=100, inertia=10),
    RigidBody([(-10, -0.1), (-10, 0.1), (10, 0.1), (10, -0.1)], 'y', mass=1e9, inertia=1e9), 
]

objects[0].set_state([0., 0., 0., 0., 0., 0.])
objects[1].set_state([0., 0.3, 0., 0., 0., 0.])
objects[2].set_state([0., 0.8, 0., 0., 0., 0.])
objects[3].set_state([0., -0.2, 0., 0., 0., 0.])


def find_contacts(objects):
    contacts = []
    for i in range(len(objects)):
        for j in range(i+1, len(objects)):
            contacts += GJK(objects[i].get_shape(), objects[j].get_shape(), plot=False)
    return contacts

plot_contacts(objects, find_contacts(objects))


from rigid_body import LCP
trajs = []
h = 0.1
for i in range(100):
    for j in range(0, 3):
        objects[j].apply_wrench(objects[j].state[:2], [0, -10 * objects[j].mass])
    LCP(objects, find_contacts(objects), h)
    trajs.append([j.step(h) for j in objects])

animate(plot_trajectoreis(objects, trajs))